In [1]:
import torch
from datasets import load_dataset
from transformers import BertForSequenceClassification, BertTokenizer, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score

In [2]:
import torch
from datasets import load_dataset
from transformers import BertForSequenceClassification, BertTokenizer, Trainer, TrainingArguments
from datasets import load_metric
from sklearn.metrics import f1_score, classification_report
import numpy as np

# Using a model fine-tuned for BOOLQ
model_name = 'lewtun/bert-base-uncased-finetuned-boolq'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)  # BOOLQ has 2 labels: true and false

c:\Users\uchau\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\uchau\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [3]:
# Using a model fine-tuned for BOOLQ
model_name = 'lewtun/bert-base-uncased-finetuned-boolq'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)  # BOOLQ has 2 labels: true and false

In [4]:
# Load the BOOLQ dataset
boolq = load_dataset('super_glue', 'boolq', trust_remote_code=True)

In [5]:
# Preprocess the dataset
def preprocess(examples):
    return tokenizer(examples['passage'], examples['question'], truncation=True, padding='max_length', max_length=128)

encoded_dataset = boolq.map(preprocess, batched=True)

# Set the format for PyTorch
encoded_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

In [6]:
# Define the metric and trainer
accuracy_metric = load_metric("accuracy")

def compute_metrics(p):
    predictions, labels = p
    preds = np.argmax(predictions, axis=1)
    acc = accuracy_metric.compute(predictions=preds, references=labels)
    f1_weighted = f1_score(labels, preds, average='weighted')
    f1_per_class = f1_score(labels, preds, average=None)
    return {'accuracy': acc['accuracy'], 'f1_weighted': f1_weighted, 'f1_per_class': f1_per_class}

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    per_device_eval_batch_size=16,   # batch size for evaluation
    do_train=False,                  # don't perform training
    do_eval=True,                    # perform evaluation
    logging_dir='./logs',            # directory for storing logs
    no_cuda=True                     # Disable GPU for debugging
)

trainer = Trainer(
    model=model,                     # the pre-trained model
    args=training_args,              # training arguments
    eval_dataset=encoded_dataset['validation'], # evaluation dataset (test set)
    compute_metrics=compute_metrics  # evaluation metric function
)

C:\Users\uchau\AppData\Local\Temp\ipykernel_4900\2195451347.py:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  accuracy_metric = load_metric("accuracy")
c:\Users\uchau\AppData\Local\Programs\Python\Python311\Lib\site-packages\datasets\load.py:759: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
c:\Users\uchau\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\training_args.py:1453: FutureWarning: usin

In [7]:
eval_result = trainer.evaluate()

  0%|          | 0/205 [00:00<?, ?it/s]

Trainer is attempting to log a value of "[0.59163273 0.7126856 ]" of type <class 'numpy.ndarray'> for key "eval/f1_per_class" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


In [8]:
acc = eval_result['eval_accuracy'] * 100
f1_weighted = eval_result['eval_f1_weighted'] * 100
f1_per_class = eval_result['eval_f1_per_class'] * 100

print(f"Accuracy: {acc:.1f}%")
print(f"Weighted F1 Score: {f1_weighted:.1f}%")
print(f"F1 Score for each class: {f1_per_class}")

Accuracy: 66.3%
Weighted F1 Score: 66.7%
F1 Score for each class: [59.16327286 71.26855952]
